In [ ]:
import numpy as np


In [ ]:
training_loss = []
exit_flag = False
while True:
    # for minibatch, _ in tqdm(dataloader):
    for minibatch, _ in tqdm(dataloader):
        # print("minibatch", minibatch, minibatch.shape)
        l = training_step.step(state, minibatch, loss)
        print("Loss:", l.item())
        training_loss.append(l.item())
        #print('ter', state["n_iter"])
        #print(state["n_iter"] % cfg.saving.checkpoint_freq)
        # just to save model
        if (
            (state["n_iter"] + 1) % cfg.saving.checkpoint_freq == 0
            or state["n_iter"] == cfg.training.n_iters - 1
        ):
            bookkeeping.save_state(state, cfg.save_location)
            print("Model saved in Iteration:", state["n_iter"] + 1)

        if (
            (state["n_iter"] + 1) % cfg.sampler.sample_freq == 0
            or state["n_iter"] == cfg.training.n_iters - 1
        ):
            state["model"].eval()
            samples, x_hist, x0_hist = sampler.sample(state["model"], n_samples, 10)
            samples = samples.reshape(n_samples, 1, 32, 32)
            #x_hist = x_hist.reshape(10, n_samples, 1, 32, 32)
            #x0_hist = x0_hist.reshape(10, n_samples, 1, 32, 32)
            state["model"].train()

            fig = plt.figure(figsize=(9, 9))  
            for i in range(n_samples):
                plt.subplot(4, 4, 1 + i)
                plt.axis("off")
                plt.imshow(np.transpose(samples[i, ...], (1,2,0)), cmap="gray")
            n_iter = state["n_iter"]
            
            saving_plot_path = os.path.join(cfg.saving.sample_plot_path, f"samples_epoch_{state['n_iter']}.png")
            plt.savefig(saving_plot_path)
            #plt.show()
            plt.close()

        state["n_iter"] += 1
        if state["n_iter"] > cfg.training.n_iters - 1:
            exit_flag = True
            break

    if exit_flag:
        break

plt.plot(training_loss)
plt.title("Training loss")
plt.savefig("SavedModels/MNIST/PNGs/training_loss.png")
plt.close()


if __name__ == "__main__":
main()
